## Using SQLModel to work with database tables

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

 - importing modules needed
 - creates a temporary db using .env

*things to do/fix*

- [X] use Postgresql as well as sqlite db
- [X] work with an existing db URL rather than create it every time
    - [X] OR create one as needed e.g. for testing

- [X] return True/False from initdb , test for success
- [X] convert all to tests
- [ ] better session management? get the 'get_session' working instead of instantiating a Session directly to capture tha function in one place 
    - [X] better way to specify the db file to work depending on context, one for dev, prod, testing like Rails?   for testing currently uses cli param
    - [X] don't use globals! middle ground - use params with globals as defaults but overridable
    - [ ] figure out how to use yield properly for 'get_session'

In progress: 

- [ ] total re-work of old "weather station" to be API specific
    - [ ] Base classes
    - [ ] constants
    - [ ] Davis
    - [ ] Onset
    - [ ] Zentra

- [ ] move previous pydantic models for readings and requests into SQLmodel
- [ ] change data structures of response and readings to fit sql
- [ ] create a collector class to combine weather table, api


TODOs for db/models.py: use pydantic validation for some fields.  here are some reminders...
```
from pydantic import field_validator
from ewxpwsdb.time_intervals import is_valid_timezone, TimeZoneStr
from ewxpwsdb.weatherstations import STATION_TYPE
# create a type for 'coordinates that enforces correct number of decimal places 
# validation on sampling_interval to be 5, 15, or 30 minutes only
```

```
# TODO validate timezone, example code
@field_validator('timezone')
@classmethod
def must_be_valid_timezone_key(cls,v: str) -> str:
    """ ensure timezone key is valid"""
    if is_valid_timezone(v):
        return(v)
    else:
        raise ValueError(f"timezone_key v is not a valid IANA timezone e.g. US/Eastern")

```

In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file, read_station_table
db_url = get_db_url()
engine = get_engine(db_url)

## create new temp database to work with

In [ ]:

init_db(engine)

In [ ]:
s = Session(engine)
objects = [
    StationType(station_type= 'DAVIS' ),
    StationType(station_type= 'SPECTRUM' ), 
    StationType(station_type= 'ONSET'),
    StationType(station_type= 'RAINWISE'),
    StationType(station_type= 'ZENTRA'),
]
s.bulk_save_objects(objects)
s.commit()

In [ ]:
station_file = '../data/test_stations.tsv'
# import, using the global 'engine' defined in database module
import_station_file(station_file)


In [ ]:
from sqlmodel import select
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()

len(stations)

In [ ]:
# attempt to add a duplicate
station_list = read_station_table(station_file)
dup_ws = WeatherStation.model_validate(station_list[2])
with Session(engine) as session:
    session.add(dup_ws)
    session.commit()


In [ ]:
our_table_names = ['WeatherStation', 'Reading', 'apiresponse', 'stationtype']
from sqlalchemy import inspect
inspector = inspect(engine)

db_tables = list(inspector.get_table_names())
db_tables

### Example SQL

In [ ]:
from sqlmodel import text

sql="select * from weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type where stationtype.station_type = 'ZENTRA';"
with Session(engine) as session:
    stmt = text(sql)
    results = session.exec(stmt)
    records = results.all()

len(records)


### Clean up

remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(WeatherStation))
        session.exec(delete(StationType))

        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()